In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import itertools
from tqdm import tqdm_notebook
from scipy.stats import spearmanr
from common import *

In [2]:
train = pd.read_csv(f'data/train.csv')
y = train[TARGETS].values
ix = np.where(train.groupby("question_body")["host"].transform("count") == 1)[0]

y_oof = pd.read_csv('oofs/siamese_albert_1_oofs.csv', index_col=0).values
y_bert = pd.read_csv('oofs/siamese_bert_6_oofs.csv', index_col=0).values
y_roberta = pd.read_csv('oofs/siamese_roberta_1_comb_oofs.csv', index_col=0).values
y_xlnet = pd.read_csv('oofs/siamese_xlnet_1_comb_oofs.csv', index_col=0).values
# y_xlnet = pd.read_csv('oofs/siamese_xlnet_2_comb_oofs.csv', index_col=0).values

In [9]:
def evaluate(y, y_pred, verbose=False, exclude=[]):
    score = 0
    for i in range(y.shape[1]):
        if i not in exclude:
            col_score = spearmanr(y[:, i], y_pred[:, i])[0]
            if verbose:
                print(TARGETS[i], np.round(col_score, 3))
            score += col_score/(y.shape[1]-len(exclude))
    return np.round(score, 3)


ds = [2, 4, 8, 16, 32, 64, None]
ws = [0, 1, 2, 4]


def scale(x, d):
    if d:
        return (x//(1/d))/d
    return x

dws4 = list(itertools.product(ds, ds, ws, ws, ws, ws))


def transform(y_oof, y_roberta, y_bert, y_xlnet, params, c):
    d_global, d_local, w_use, w_roberta, w_bert, w_xlnet = params
    y_temp = scale(y_oof[:, c], d_local)*w_use +\
                 scale(y_roberta[:, c], d_local)*w_roberta +\
                 scale(y_bert[:, c], d_local)*w_bert +\
                 scale(y_xlnet[:, c], d_local)*w_xlnet
    y_temp /= w_use + w_roberta + w_bert + w_xlnet
    y_temp = scale(y_temp, d_global)
    return y_temp


def f(c):
    max_score = spearmanr(y[:, c], y_roberta[:, c])[0]
    best_index = -1
    for i, params in enumerate(dws4):
        y_temp = transform(y_oof, y_roberta, y_bert, y_xlnet, params, c)
        score = spearmanr(y[:, c], y_temp)[0]
        if score > max_score:
            max_score = score
            best_index = i
            
    return best_index

In [4]:
import multiprocessing

pool = multiprocessing.Pool(15)
out = pool.map(f, range(30))

In [6]:
y_combined = np.zeros(y.shape)

for c in range(30):
    print(c, TARGETS[c], dws4[out[c]])
    y_combined[:, c] = transform(y_oof, y_roberta, y_bert, y_xlnet, dws4[out[c]], c)

0 question_asker_intent_understanding (64, 32, 1, 1, 1, 2)
1 question_body_critical (32, None, 4, 1, 4, 1)
2 question_conversational (4, None, 2, 4, 1, 2)
3 question_expect_short_answer (32, None, 2, 1, 1, 1)
4 question_fact_seeking (8, 32, 1, 1, 2, 4)
5 question_has_commonly_accepted_answer (4, None, 4, 2, 1, 4)
6 question_interestingness_others (32, None, 2, 0, 1, 2)
7 question_interestingness_self (16, 32, 2, 1, 4, 4)
8 question_multi_intent (16, 4, 1, 1, 4, 2)
9 question_not_really_a_question (16, 8, 2, 0, 1, 1)
10 question_opinion_seeking (16, 64, 2, 2, 1, 1)
11 question_type_choice (8, 8, 1, 2, 2, 2)
12 question_type_compare (4, None, 0, 4, 1, 4)
13 question_type_consequence (4, 8, 0, 1, 0, 4)
14 question_type_definition (8, 8, 0, 4, 1, 0)
15 question_type_entity (4, 64, 0, 4, 1, 2)
16 question_type_instructions (8, 16, 1, 2, 1, 2)
17 question_type_procedure (64, 64, 1, 2, 0, 2)
18 question_type_reason_explanation (32, 64, 1, 4, 2, 4)
19 question_type_spelling (16, 64, 0, 0, 4, 1

In [10]:
evaluate(y, y_combined, True)

question_asker_intent_understanding 0.402
question_body_critical 0.679
question_conversational 0.513
question_expect_short_answer 0.323
question_fact_seeking 0.381
question_has_commonly_accepted_answer 0.479
question_interestingness_others 0.373
question_interestingness_self 0.52
question_multi_intent 0.606
question_not_really_a_question 0.201
question_opinion_seeking 0.496
question_type_choice 0.771
question_type_compare 0.566
question_type_consequence 0.333
question_type_definition 0.669
question_type_entity 0.612
question_type_instructions 0.791
question_type_procedure 0.384
question_type_reason_explanation 0.684
question_type_spelling 0.563
question_well_written 0.539
answer_helpful 0.272
answer_level_of_information 0.476
answer_plausible 0.174
answer_relevance 0.191
answer_satisfaction 0.38
answer_type_instructions 0.774
answer_type_procedure 0.327
answer_type_reason_explanation 0.697
answer_well_written 0.25


0.481

In [11]:
[dws4[out[c]] for c in range(30)]

[(64, 32, 1, 1, 1, 2),
 (32, None, 4, 1, 4, 1),
 (4, None, 2, 4, 1, 2),
 (32, None, 2, 1, 1, 1),
 (8, 32, 1, 1, 2, 4),
 (4, None, 4, 2, 1, 4),
 (32, None, 2, 0, 1, 2),
 (16, 32, 2, 1, 4, 4),
 (16, 4, 1, 1, 4, 2),
 (16, 8, 2, 0, 1, 1),
 (16, 64, 2, 2, 1, 1),
 (8, 8, 1, 2, 2, 2),
 (4, None, 0, 4, 1, 4),
 (4, 8, 0, 1, 0, 4),
 (8, 8, 0, 4, 1, 0),
 (4, 64, 0, 4, 1, 2),
 (8, 16, 1, 2, 1, 2),
 (64, 64, 1, 2, 0, 2),
 (32, 64, 1, 4, 2, 4),
 (16, 64, 0, 0, 4, 1),
 (None, None, 1, 2, 4, 2),
 (16, 8, 2, 4, 1, 1),
 (16, 32, 0, 2, 1, 1),
 (16, 32, 1, 4, 1, 1),
 (64, 64, 1, 2, 0, 1),
 (32, 32, 1, 4, 1, 4),
 (16, 32, 1, 2, 1, 1),
 (64, 64, 0, 4, 2, 1),
 (64, None, 2, 4, 1, 2),
 (32, None, 0, 1, 2, 2)]